In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

bgg_Category = pd.read_csv('scraped/bgg_Category.csv')
bgg_GameFamily = pd.read_csv('scraped/bgg_GameFamily.csv')
bgg_GameItem = pd.read_csv('scraped/bgg_GameItem.csv')
bgg_GameType = pd.read_csv('scraped/bgg_GameType.csv')
bgg_Mechanic = pd.read_csv('scraped/bgg_Mechanic.csv')
bgg_Person = pd.read_csv('scraped/bgg_Person.csv')
bgg_Publisher = pd.read_csv('scraped/bgg_Publisher.csv')

In [57]:
drop=[
'bga_id',
'dbpedia_id',
'luding_id',
'spielen_id',
'wikidata_id',
'wikipedia_id']

In [58]:
predictor = [
#'name',
'year',
'bgg_id',
'designer',
]
response_vars = [
'num_votes',
'avg_rating',
#'stddev_rating',
#'rank',
'bayes_rating',
"novelty_norm",
]
actor_covariates = [
#"is.man"
]
game_covariates = [
 'complexity',
 'game_type',
 'min_players',
 'max_players',
 'category',
# 'mechanic',
# 'publisher',
# 'min_players_rec',
# 'max_players_rec',
# 'min_players_best',
# 'max_players_best',
# 'min_age',
# 'min_age_rec',
# 'min_time',
# 'max_time',
# 'cooperative',
# 'family',
# 'language_dependency',
# 'artist',
# # filter out ?
# 'compilation',
# 'compilation_of',
# 'implementation',
# 'integration',
#
]

<h1>Cleaning</h1>

In [59]:
#df_core = bgg_GameItem.copy()[core+game_covariates+response]
df_core = bgg_GameItem.drop(columns=drop).copy()
core = df_core[df_core.avg_rating.notna()] # at least 1 ratings
core = core[core.bayes_rating.notna()] # at least 30 ratings
core = core[(core.year >= 1990) & (core.year < 2024)] # only games after key:  1990 <= year < 2024
core.designer = core.designer.str.split(",")
core [core.designer.map(lambda x: "3" in x, na_action="ignore").fillna(False)] = pd.NA # 3 is designer="uncredited"
core = core[core.designer.notna()] # must include designer
core = core[core.designer.map(len) <= 16] # huge groups are rare and lead to effects
core = core[core.mechanic.notna()] # must include mechanics
core.mechanic = core.mechanic.str.split(",").map(set).map(sorted).map(tuple)
core = core.sort_values("year").reset_index(drop=True)
core.head(2)

,bgg_id,name,year,game_type,designer,artist,publisher,min_players,max_players,min_players_rec,...,family,implementation,integration,rank,num_votes,avg_rating,stddev_rating,bayes_rating,complexity,language_dependency
0,18330.0,Dragon Noir 1: L'Exil,1990.0,4664,[80],"4911,11802,11992",116,2.0,7.0,2.0,...,"7005,3273",NaN,NaN,13062.0,54.0,6.95648,1.14795,5.53777,2.5000,NaN
1,2379.0,Guesstures,1990.0,5498,"[8083, 11050]",NaN,"3115,858,51,20,28,4653",4.0,99.0,4.0,...,"10714,6042",NaN,NaN,6188.0,1702.0,5.90289,1.53140,5.66313,1.1802,NaN


In [60]:
"""by_id = core.sort_values("year").reset_index(drop=True)
by_id.designer = by_id.designer.str.split(",")
by_id = by_id.explode("designer")
by_id["event_type"] = "event"
main = ["bgg_id","year","designer","event_type"]
by_id = by_id[main+[x for x in by_id if x not in main]]
by_id.head(2)"""

'by_id = core.sort_values("year").reset_index(drop=True)\nby_id.designer = by_id.designer.str.split(",")\nby_id = by_id.explode("designer")\nby_id["event_type"] = "event"\nmain = ["bgg_id","year","designer","event_type"]\nby_id = by_id[main+[x for x in by_id if x not in main]]\nby_id.head(2)'

<h1>Compute Novelties

In [61]:
#step 1
configurations = core.mechanic.unique()
lut = {v:k for k,v in enumerate(configurations)}
dummies = pd.Series(configurations).explode().str.get_dummies().groupby(level=0).sum()
dummies

,2001,2002,2003,2004,2005,2007,2008,2009,2010,2011,...,3007,3096,3097,3098,3099,3100,3101,3102,3103,3104
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9452,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9453,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9455,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#step 2: euclidean distances between each pair of configurations
from sklearn.metrics import pairwise_distances
dist_matrix = pd.DataFrame(pairwise_distances(dummies),index=dummies.index,columns=dummies.index)
dist_matrix = dist_matrix/dist_matrix.max(None)
dist_matrix.head(2)

,0,1,2,3,4,5,6,7,8,9,...,9447,9448,9449,9450,9451,9452,9453,9454,9455,9456
0,0.000000,0.361158,0.442326,0.442326,0.294884,0.294884,0.361158,0.255377,0.361158,0.361158,...,0.294884,0.361158,0.442326,0.510754,0.417029,0.361158,0.390095,0.442326,0.390095,0.417029
1,0.361158,0.000000,0.489010,0.531610,0.361158,0.361158,0.466252,0.329690,0.417029,0.417029,...,0.361158,0.417029,0.489010,0.551677,0.466252,0.417029,0.442326,0.442326,0.442326,0.466252


In [63]:
#step 3: novelty = average of distances of the focal game to all of the games that existed at that time
from tqdm import tqdm
import numpy as np

def min_max(df):
    return (df-df.min(None))/(df.max(None)-df.min(None))

mechanics_by_year = core[["bgg_id","year","mechanic"]].copy().reset_index(drop=True)
mechanics_by_year["configuration_id"] = mechanics_by_year.mechanic.map(lambda x :lut[x])
mechanics_by_year = mechanics_by_year.set_index("year").sort_index()
novelty = mechanics_by_year.reset_index()
for i,ix in enumerate(mechanics_by_year.index):
    focal = novelty.iloc[i]["configuration_id"]
    previous = mechanics_by_year.loc[:ix,"configuration_id"]
    novelty.loc[i,"novelty"] = dist_matrix.to_numpy()[previous.values,focal].mean()
novelty["novelty_norm"] = novelty[["novelty"]].apply(min_max).convert_dtypes()
core_vars = core.merge(novelty,on=["year","bgg_id"])

<h1> Add dummy events


In [64]:
# actor_dummy: who could participate in events actor dummies
dummy_actor = core.explode(column="designer").groupby("designer")["year"].min().reset_index()
dummy_actor["event_type"]= "add_actor"
dummy_actor.head(2)

,designer,year,event_type
0,1,1990.0,add_actor
1,10,1996.0,add_actor


In [65]:
# dummy events for setting gender on actors
person_gendered = pd.read_csv('bgg_person_gendered.csv')
person_gendered.bgg_id = person_gendered.bgg_id.astype(str)
dummy_gender = pd.merge(pd.Series(core.designer.explode().unique(),name="designer"),person_gendered,left_on="designer",right_on="bgg_id")
dummy_gender = dummy_gender.drop(columns=["bgg_id","name"])
dummy_gender = dummy_gender.rename({"is.man":"weight"},axis=1)
dummy_gender["event_type"]="add_gender"
dummy_gender.head(2)

,designer,weight,event_type
0,80,-1,add_gender
1,8083,1,add_gender


In [66]:
# actor_dummy: who could participate in events actor dummies
dummy_actor_by_year = core.explode(column="designer").groupby("designer")["year"].min().reset_index()
interval = np.arange(dummy_actor_by_year.year.min(),dummy_actor_by_year.year.max()+1)
dummy_tenure = dummy_actor_by_year.set_index("designer").apply(lambda x:interval-x.year,result_type="expand",axis=1).set_axis(interval,axis=1).T
dummy_tenure = dummy_tenure.stack().clip(lower=-1).reset_index().set_axis(["year","designer","weight"],axis=1).drop_duplicates(subset=["designer","weight"])#.dropna(subset="weight")#.replace(-1,pd.NA)
dummy_tenure["event_type"]="add_tenure"
dummy_tenure


,year,designer,weight,event_type
0,1990.0,1,0.0,add_tenure
1,1990.0,10,-1.0,add_tenure
2,1990.0,1000,-1.0,add_tenure
3,1990.0,100012,-1.0,add_tenure
4,1990.0,100025,-1.0,add_tenure
...,...,...,...,...
334317,2023.0,99909,5.0,add_tenure
334318,2023.0,99951,6.0,add_tenure
334319,2023.0,99957,6.0,add_tenure
334320,2023.0,99958,6.0,add_tenure


<h1> Export AND set outcome/target


In [67]:
import csv
for target in response_vars:
    outcome = core_vars.copy()
    outcome = core_vars.rename({target:"weight"},axis=1,errors="raise")
    cols = predictor  + game_covariates + actor_covariates + ["event_type","weight"]
    outcome["event_type"] = "event"
    eventlist = pd.concat([outcome,dummy_actor,dummy_gender,dummy_tenure],axis=0)
    eventlist = eventlist.sort_values(["year","event_type"],ascending=(True,True)).reset_index(drop=True).convert_dtypes()
    numeric = eventlist.select_dtypes("number").columns 
    eventlist.loc[:,numeric] = eventlist.loc[:,numeric].fillna(0)
    eventlist = eventlist.explode(column="designer")
    eventlist = eventlist.sort_values(["year","event_type"],ascending=(True,True)).reset_index(drop=True)
    eventlist[cols].to_csv("../RHOM_input/bgg_"+target+".csv",index=False,quoting=csv.QUOTE_NONNUMERIC)